# GANITE - 生成对抗网络遇上因果推断

---

## 学习目标

完成本练习后，你将能够：

1. 理解 GANITE 的核心思想：用 GAN 生成反事实结果
2. 掌握 GANITE 的两阶段架构（反事实块 + ITE 块）
3. 理解为什么 GAN 适合处理因果推断的反事实问题
4. 实现简化版 GANITE 并在实验中评估
5. 比较 GANITE 与 TARNet/DragonNet 的优劣

---

## 真实场景：医疗影像诊断的「如果」

想象你是一家医院的 AI 研究员，正在研究一种新型癌症治疗方案：

> **场景**：
> - 患者 A 接受了新疗法，3 个月后肿瘤缩小了 40%
> - 但我们永远无法知道：**如果他没接受新疗法，会怎样？**

这就是因果推断的核心问题——**反事实无法观测**。

### 传统方法的局限

- **TARNet/DragonNet**：学习 $\mu_0(x)$ 和 $\mu_1(x)$，但本质是「预测」
- **问题**：预测值可能不在合理的分布范围内

### GANITE 的创新思路

> **既然反事实是「不存在的数据」，那我们就用 GAN 来「生成」它！**

就像 DeepFake 可以生成「从未存在的照片」，GANITE 生成「从未发生的结果」。

### 生动比喻：平行宇宙生成器

想象 GANITE 是一台「平行宇宙模拟器」：

```
[现实宇宙] 患者 A 接受治疗 → 观测到结果 Y(1)
          │
    ┌─────┴─────┐
    │  GANITE   │ ← 生成器
    └─────┬─────┘
          │
[模拟宇宙] 如果患者 A 没接受治疗 → 生成结果 Ŷ(0)
```

判别器的任务是：**分辨哪些结果是「真实观测」，哪些是「模拟生成」**。

---

## 核心概念：GAN 的基本原理

### GAN 快速回顾

生成对抗网络 (Generative Adversarial Network) 包含两个网络：

| 角色 | 任务 | 类比 |
|------|------|------|
| **生成器 (G)** | 从噪声生成假数据 | 造假者 |
| **判别器 (D)** | 区分真假数据 | 鉴定师 |

两者玩「猫鼠游戏」：
- G 努力让 D 分辨不出真假
- D 努力提高鉴别能力

### GAN 的目标函数

$$\min_G \max_D V(D, G) = \mathbb{E}_{x \sim p_{data}}[\log D(x)] + \mathbb{E}_{z \sim p_z}[\log(1 - D(G(z)))]$$

**直观理解**：
- $D(x)$: 判别器认为 $x$ 是真实数据的概率
- $G(z)$: 生成器从噪声 $z$ 生成的假数据
- 判别器想让 $D(x) \to 1$，$D(G(z)) \to 0$
- 生成器想让 $D(G(z)) \to 1$

---

## GANITE 架构详解

### 核心创新：两阶段生成

GANITE 由两个 GAN 块组成：

```
┌─────────────────────────────────────────────────────────────┐
│                     GANITE 整体架构                          │
├─────────────────────────────────────────────────────────────┤
│                                                             │
│   ┌───────────────────────────────────────┐                │
│   │        Block 1: 反事实生成块           │                │
│   │                                       │                │
│   │   输入: (X, T, Y_factual)             │                │
│   │   输出: Ŷ_counterfactual              │                │
│   │                                       │                │
│   │   G_cf: 生成反事实结果                 │                │
│   │   D_cf: 判别生成的反事实是否合理        │                │
│   └───────────────────────────────────────┘                │
│                       │                                    │
│                       ▼                                    │
│   ┌───────────────────────────────────────┐                │
│   │        Block 2: ITE 推断块            │                │
│   │                                       │                │
│   │   输入: (X, Ŷ(0), Ŷ(1)) ← 来自 Block1 │                │
│   │   输出: τ̂(X) = ITE 估计               │                │
│   │                                       │                │
│   │   G_ite: 生成 ITE 估计                │                │
│   │   D_ite: 判别 ITE 是否与数据一致       │                │
│   └───────────────────────────────────────┘                │
│                                                             │
└─────────────────────────────────────────────────────────────┘
```

### 为什么需要两个块？

1. **Block 1 (反事实块)**：解决「生成缺失的潜在结果」
   - 如果 T=1，生成 Ŷ(0)
   - 如果 T=0，生成 Ŷ(1)

2. **Block 2 (ITE 块)**：解决「从完整结果估计处理效应」
   - 输入：完整的 (Ŷ(0), Ŷ(1))（Block 1 填补后）
   - 输出：更稳健的 ITE 估计

### Block 1 的详细设计

**生成器 $G_{cf}$**：
$$\tilde{Y}_{cf} = G_{cf}(X, T, Y, Z_{cf})$$

- 输入：特征 X、处理 T、观测结果 Y、随机噪声 $Z_{cf}$
- 输出：反事实结果（T=1 时生成 Y(0)，T=0 时生成 Y(1)）

**判别器 $D_{cf}$**：
$$D_{cf}(X, T, Y, \tilde{Y}_{cf}) \in [0, 1]$$

- 判断：生成的反事实 $\tilde{Y}_{cf}$ 是否「看起来真实」

### Block 2 的详细设计

**生成器 $G_{ite}$**：
$$\hat{\tau}(X) = G_{ite}(X)$$

**注意**：ITE 生成器是确定性的，不需要随机噪声

**判别器 $D_{ite}$**：
$$D_{ite}(X, \hat{Y}(0), \hat{Y}(1))$$

- 判断：基于 ITE 估计重构的结果是否与真实观测一致

---

## GANITE 损失函数

### Block 1: 反事实块损失

**判别器损失**（使用观测值作为监督信号）：
$$\mathcal{L}_{D_{cf}} = -\mathbb{E}[\log D_{cf}(X, T, Y, Y)] - \mathbb{E}[\log(1 - D_{cf}(X, T, Y, \tilde{Y}_{cf}))]$$

**说明**：判别器学习区分：
- 真实样本：观测到的 $(X, T, Y, Y)$ - 第四个位置用观测值作为代理
- 假样本：生成的 $(X, T, Y, \tilde{Y}_{cf})$

**生成器损失**：
$$\mathcal{L}_{G_{cf}} = -\mathbb{E}[\log D_{cf}(X, T, Y, \tilde{Y}_{cf})] + \lambda \cdot \mathcal{L}_{supervised}$$

其中监督损失（利用观测值）：
$$\mathcal{L}_{supervised} = \mathbb{E}[(Y - \hat{Y}_{factual})^2]$$

这里 $\hat{Y}_{factual}$ 是根据 T 选择的：
- 当 $T=1$ 时，$\hat{Y}_{factual} = \hat{Y}(1)$（应该接近观测的 Y）
- 当 $T=0$ 时，$\hat{Y}_{factual} = \hat{Y}(0)$（应该接近观测的 Y）

### Block 2: ITE 块损失

**判别器损失**：
$$\mathcal{L}_{D_{ite}} = -\mathbb{E}[\log D_{ite}(X, \hat{Y}(0), \hat{Y}(1))] - \mathbb{E}[\log(1 - D_{ite}(X, \hat{Y}_0^{ite}, \hat{Y}_1^{ite}))]$$

其中 $\hat{Y}_0^{ite} = \hat{Y}(1) - \hat{\tau}(X)$，$\hat{Y}_1^{ite} = \hat{Y}(0) + \hat{\tau}(X)$

**生成器损失**：
$$\mathcal{L}_{G_{ite}} = -\mathbb{E}[\log D_{ite}(X, \hat{Y}_0^{ite}, \hat{Y}_1^{ite})]$$

**一致性约束**：生成的 ITE 必须与 Block 1 的输出一致

---

In [ ]:
# 环境准备
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from typing import Tuple, Dict
import warnings
warnings.filterwarnings('ignore')

# 设置随机种子
np.random.seed(42)
torch.manual_seed(42)

# 颜色方案
COLORS = {
    'primary': '#2D9CDB',
    'success': '#27AE60',
    'danger': '#EB5757',
    'warning': '#F2994A',
    'info': '#9B51E0',
}

print(f"PyTorch 版本: {torch.__version__}")
print("环境准备完成！")

---

## Part 1: 数据生成

### 场景：个性化医疗治疗效果预测

In [ ]:
def generate_medical_data(n_samples: int = 3000, seed: int = 42) -> Tuple:
    """
    生成模拟医疗数据
    
    场景：癌症治疗效果预测
    - X: 患者特征（年龄、基因表达、肿瘤大小等）
    - T: 是否接受新疗法
    - Y: 6 个月后肿瘤缩小比例
    """
    np.random.seed(seed)
    
    # 患者特征 (10 维)
    X = np.random.randn(n_samples, 10)
    # X[:, 0]: 年龄标准化
    # X[:, 1]: 基因标记 A
    # X[:, 2]: 基因标记 B
    # X[:, 3]: 肿瘤初始大小
    # X[:, 4-9]: 其他生物标志物
    
    # 治疗分配（非随机，医生倾向给年轻、基因好的患者用新疗法）
    propensity = 1 / (1 + np.exp(-(
        -0.5 * X[:, 0] +   # 年轻患者更可能接受
        0.8 * X[:, 1] +    # 基因标记 A 阳性更可能接受
        0.3 * X[:, 2]      # 基因标记 B 有轻微影响
    )))
    T = np.random.binomial(1, propensity, n_samples)
    
    # 潜在结果 - 复杂的非线性关系
    noise = np.random.randn(n_samples) * 0.3
    
    # Y(0): 标准疗法的效果
    Y0 = (
        0.5 +
        0.2 * np.tanh(X[:, 0]) +      # 年龄的非线性效应
        0.15 * X[:, 1] +              # 基因 A 的线性效应
        0.1 * X[:, 3] ** 2 +          # 肿瘤大小的二次效应
        noise
    )
    
    # 异质性处理效应 - 不同患者对新疗法反应不同
    tau = (
        0.3 +                                    # 基础效应
        0.4 * (X[:, 1] > 0).astype(float) +      # 基因 A 阳性者效应更大
        0.2 * np.tanh(-X[:, 0]) +                # 年轻患者效应更大
        0.15 * X[:, 2]                           # 基因 B 的调节效应
    )
    
    # Y(1): 新疗法的效果
    Y1 = Y0 + tau
    
    # 观测结果
    Y = np.where(T == 1, Y1, Y0)
    
    return X, T, Y, Y0, Y1, tau

# 生成数据
X, T, Y, Y0, Y1, true_tau = generate_medical_data(n_samples=3000)

print("数据生成完成！")
print(f"样本数: {len(X)}")
print(f"特征维度: {X.shape[1]}")
print(f"治疗组比例: {T.mean():.2%}")
print(f"\n真实效应统计:")
print(f"  ATE: {true_tau.mean():.4f}")
print(f"  CATE 标准差: {true_tau.std():.4f}")
print(f"  CATE 范围: [{true_tau.min():.4f}, {true_tau.max():.4f}]")

---

## Part 2: GANITE 实现

### 2.1 反事实生成块 (Counterfactual Block)

In [ ]:
class CounterfactualGenerator(nn.Module):
    """
    反事实生成器 G_cf
    
    输入: (X, T, Y, Z) 其中 Z 是随机噪声
    输出: 反事实结果 Ŷ_cf
    """
    
    def __init__(self, input_dim: int, hidden_dim: int = 64, noise_dim: int = 10):
        super().__init__()
        
        self.noise_dim = noise_dim
        
        # 输入: X (input_dim) + T (1) + Y (1) + Z (noise_dim)
        total_input = input_dim + 1 + 1 + noise_dim
        
        self.net = nn.Sequential(
            nn.Linear(total_input, hidden_dim),
            nn.LeakyReLU(0.2),
            nn.BatchNorm1d(hidden_dim),
            nn.Linear(hidden_dim, hidden_dim),
            nn.LeakyReLU(0.2),
            nn.BatchNorm1d(hidden_dim),
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.LeakyReLU(0.2),
            nn.Linear(hidden_dim // 2, 1)
        )
    
    def forward(self, x: torch.Tensor, t: torch.Tensor, y: torch.Tensor) -> torch.Tensor:
        batch_size = x.shape[0]
        
        # 生成随机噪声
        z = torch.randn(batch_size, self.noise_dim, device=x.device)
        
        # 拼接输入
        t = t.view(-1, 1) if len(t.shape) == 1 else t
        y = y.view(-1, 1) if len(y.shape) == 1 else y
        
        inputs = torch.cat([x, t, y, z], dim=1)
        
        return self.net(inputs)


class CounterfactualDiscriminator(nn.Module):
    """
    反事实判别器 D_cf
    
    输入: (X, T, Y_factual, Y_counterfactual)
    输出: 概率（反事实是否真实）
    """
    
    def __init__(self, input_dim: int, hidden_dim: int = 64):
        super().__init__()
        
        # 输入: X + T + Y_factual + Y_cf
        total_input = input_dim + 1 + 1 + 1
        
        self.net = nn.Sequential(
            nn.Linear(total_input, hidden_dim),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3),
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3),
            nn.Linear(hidden_dim // 2, 1),
            nn.Sigmoid()
        )
    
    def forward(self, x: torch.Tensor, t: torch.Tensor, 
                y_factual: torch.Tensor, y_cf: torch.Tensor) -> torch.Tensor:
        t = t.view(-1, 1) if len(t.shape) == 1 else t
        y_factual = y_factual.view(-1, 1) if len(y_factual.shape) == 1 else y_factual
        y_cf = y_cf.view(-1, 1) if len(y_cf.shape) == 1 else y_cf
        
        inputs = torch.cat([x, t, y_factual, y_cf], dim=1)
        
        return self.net(inputs)


print("反事实块定义完成！")

### 2.2 ITE 推断块 (ITE Inference Block)

In [ ]:
class ITEGenerator(nn.Module):
    """
    ITE 生成器 G_ite
    
    输入: X
    输出: ITE 估计 τ̂(X)
    """
    
    def __init__(self, input_dim: int, hidden_dim: int = 64):
        super().__init__()
        
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ELU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ELU(),
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ELU(),
            nn.Linear(hidden_dim // 2, 1)
        )
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.net(x)


class ITEDiscriminator(nn.Module):
    """
    ITE 判别器 D_ite
    
    输入: (X, Ŷ(0), Ŷ(1))
    输出: 概率（ITE 估计是否与数据一致）
    """
    
    def __init__(self, input_dim: int, hidden_dim: int = 64):
        super().__init__()
        
        # 输入: X + Y0 + Y1
        total_input = input_dim + 2
        
        self.net = nn.Sequential(
            nn.Linear(total_input, hidden_dim),
            nn.LeakyReLU(0.2),
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.LeakyReLU(0.2),
            nn.Linear(hidden_dim // 2, 1),
            nn.Sigmoid()
        )
    
    def forward(self, x: torch.Tensor, y0: torch.Tensor, y1: torch.Tensor) -> torch.Tensor:
        y0 = y0.view(-1, 1) if len(y0.shape) == 1 else y0
        y1 = y1.view(-1, 1) if len(y1.shape) == 1 else y1
        
        inputs = torch.cat([x, y0, y1], dim=1)
        
        return self.net(inputs)


print("ITE 块定义完成！")

### 2.3 完整 GANITE 模型

In [ ]:
class GANITE(nn.Module):
    """
    完整的 GANITE 模型
    
    包含两个 GAN 块：
    1. 反事实生成块：生成缺失的潜在结果
    2. ITE 推断块：估计个体处理效应
    """
    
    def __init__(self, input_dim: int, hidden_dim: int = 64, noise_dim: int = 10):
        super().__init__()
        
        self.input_dim = input_dim
        
        # Block 1: 反事实块
        self.cf_generator = CounterfactualGenerator(input_dim, hidden_dim, noise_dim)
        self.cf_discriminator = CounterfactualDiscriminator(input_dim, hidden_dim)
        
        # Block 2: ITE 块
        self.ite_generator = ITEGenerator(input_dim, hidden_dim)
        self.ite_discriminator = ITEDiscriminator(input_dim, hidden_dim)
    
    def generate_counterfactual(self, x: torch.Tensor, t: torch.Tensor, 
                                 y: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
        """
        生成反事实结果
        
        Returns:
            (y0_hat, y1_hat): 完整的潜在结果估计
        """
        # 生成反事实
        y_cf = self.cf_generator(x, t, y)
        
        # 组合观测值和生成值
        t = t.view(-1, 1)
        y = y.view(-1, 1)
        
        # T=1 时：Y_factual = Y(1), Y_cf = Ŷ(0)
        # T=0 时：Y_factual = Y(0), Y_cf = Ŷ(1)
        y0_hat = torch.where(t == 1, y_cf, y)
        y1_hat = torch.where(t == 1, y, y_cf)
        
        return y0_hat, y1_hat
    
    def predict_ite(self, x: torch.Tensor) -> torch.Tensor:
        """
        预测 ITE
        """
        return self.ite_generator(x)
    
    def forward(self, x: torch.Tensor, t: torch.Tensor, 
                y: torch.Tensor) -> Dict[str, torch.Tensor]:
        """
        完整前向传播
        """
        # Block 1: 生成反事实
        y0_hat, y1_hat = self.generate_counterfactual(x, t, y)
        
        # Block 2: 估计 ITE
        ite_hat = self.ite_generator(x)
        
        return {
            'y0_hat': y0_hat,
            'y1_hat': y1_hat,
            'ite_hat': ite_hat
        }


# 测试模型
model = GANITE(input_dim=10)
X_tensor = torch.FloatTensor(X[:10])
T_tensor = torch.FloatTensor(T[:10])
Y_tensor = torch.FloatTensor(Y[:10])

outputs = model(X_tensor, T_tensor, Y_tensor)
print("GANITE 模型测试通过！")
print(f"Y0_hat 形状: {outputs['y0_hat'].shape}")
print(f"Y1_hat 形状: {outputs['y1_hat'].shape}")
print(f"ITE_hat 形状: {outputs['ite_hat'].shape}")

---

## Part 3: 训练 GANITE

### 3.1 训练循环

In [ ]:
def train_ganite(
    model: GANITE,
    X: np.ndarray,
    T: np.ndarray,
    Y: np.ndarray,
    n_epochs: int = 300,
    batch_size: int = 128,
    lr_g: float = 1e-3,
    lr_d: float = 1e-3,
    alpha: float = 1.0,  # 监督损失权重
    verbose: bool = True
) -> Dict:
    """
    训练 GANITE 模型
    """
    # 数据准备
    X_tensor = torch.FloatTensor(X)
    T_tensor = torch.FloatTensor(T)
    Y_tensor = torch.FloatTensor(Y)
    
    dataset = TensorDataset(X_tensor, T_tensor, Y_tensor)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    
    # 优化器
    opt_cf_g = optim.Adam(model.cf_generator.parameters(), lr=lr_g, betas=(0.5, 0.999))
    opt_cf_d = optim.Adam(model.cf_discriminator.parameters(), lr=lr_d, betas=(0.5, 0.999))
    opt_ite_g = optim.Adam(model.ite_generator.parameters(), lr=lr_g, betas=(0.5, 0.999))
    opt_ite_d = optim.Adam(model.ite_discriminator.parameters(), lr=lr_d, betas=(0.5, 0.999))
    
    # 损失函数
    bce_loss = nn.BCELoss()
    mse_loss = nn.MSELoss()
    
    # 训练历史
    history = {
        'cf_g_loss': [], 'cf_d_loss': [],
        'ite_g_loss': [], 'ite_d_loss': []
    }
    
    for epoch in range(n_epochs):
        epoch_losses = {k: 0.0 for k in history.keys()}
        n_batches = 0
        
        for batch_x, batch_t, batch_y in dataloader:
            batch_size = batch_x.shape[0]
            
            # ========== Block 1: 反事实块训练 ==========
            
            # --- 训练判别器 D_cf ---
            opt_cf_d.zero_grad()
            
            # 生成反事实
            with torch.no_grad():
                y_cf_fake = model.cf_generator(batch_x, batch_t, batch_y)
            
            # 真实标签和假标签
            real_labels = torch.ones(batch_size, 1) * 0.9  # label smoothing
            fake_labels = torch.zeros(batch_size, 1) + 0.1
            
            # 判别器对真实样本的输出（使用观测的 Y 作为"真实"反事实的代理）
            # 注意：这里简化处理，实际 GANITE 使用更复杂的监督信号
            d_real = model.cf_discriminator(batch_x, batch_t, batch_y, batch_y.view(-1, 1))
            d_fake = model.cf_discriminator(batch_x, batch_t, batch_y, y_cf_fake)
            
            d_loss = bce_loss(d_real, real_labels) + bce_loss(d_fake, fake_labels)
            d_loss.backward()
            opt_cf_d.step()
            
            # --- 训练生成器 G_cf ---
            opt_cf_g.zero_grad()
            
            y_cf_fake = model.cf_generator(batch_x, batch_t, batch_y)
            d_fake = model.cf_discriminator(batch_x, batch_t, batch_y, y_cf_fake)
            
            # 生成器损失 = 对抗损失 + 监督损失
            g_adv_loss = bce_loss(d_fake, real_labels)
            
            # 监督损失：生成的结果应该接近观测值（当生成的是 factual 时）
            y0_hat, y1_hat = model.generate_counterfactual(batch_x, batch_t, batch_y)
            y_pred = torch.where(batch_t.view(-1, 1) == 1, y1_hat, y0_hat)
            supervised_loss = mse_loss(y_pred, batch_y.view(-1, 1))
            
            g_loss = g_adv_loss + alpha * supervised_loss
            g_loss.backward()
            opt_cf_g.step()
            
            # ========== Block 2: ITE 块训练 ==========
            
            # 获取填补后的完整结果
            with torch.no_grad():
                y0_hat, y1_hat = model.generate_counterfactual(batch_x, batch_t, batch_y)
            
            # --- 训练 ITE 判别器 ---
            opt_ite_d.zero_grad()
            
            # 从 ITE 生成器得到 ITE
            with torch.no_grad():
                ite_fake = model.ite_generator(batch_x)
            
            # 基于 ITE 重构结果
            y0_from_ite = y1_hat - ite_fake
            y1_from_ite = y0_hat + ite_fake
            
            # 真实：来自 Block 1 的填补结果
            d_real_ite = model.ite_discriminator(batch_x, y0_hat, y1_hat)
            # 假：从 ITE 重构的结果
            d_fake_ite = model.ite_discriminator(batch_x, y0_from_ite, y1_from_ite)
            
            d_ite_loss = bce_loss(d_real_ite, real_labels) + bce_loss(d_fake_ite, fake_labels)
            d_ite_loss.backward()
            opt_ite_d.step()
            
            # --- 训练 ITE 生成器 ---
            opt_ite_g.zero_grad()
            
            ite_fake = model.ite_generator(batch_x)
            y0_from_ite = y1_hat - ite_fake
            y1_from_ite = y0_hat + ite_fake
            
            d_fake_ite = model.ite_discriminator(batch_x, y0_from_ite, y1_from_ite)
            
            g_ite_loss = bce_loss(d_fake_ite, real_labels)
            g_ite_loss.backward()
            opt_ite_g.step()
            
            # 记录损失
            epoch_losses['cf_g_loss'] += g_loss.item()
            epoch_losses['cf_d_loss'] += d_loss.item()
            epoch_losses['ite_g_loss'] += g_ite_loss.item()
            epoch_losses['ite_d_loss'] += d_ite_loss.item()
            n_batches += 1
        
        # 记录平均损失
        for k in history.keys():
            history[k].append(epoch_losses[k] / n_batches)
        
        if verbose and (epoch + 1) % 50 == 0:
            print(f"Epoch {epoch+1}/{n_epochs}")
            print(f"  CF Block - G: {history['cf_g_loss'][-1]:.4f}, D: {history['cf_d_loss'][-1]:.4f}")
            print(f"  ITE Block - G: {history['ite_g_loss'][-1]:.4f}, D: {history['ite_d_loss'][-1]:.4f}")
    
    return history

In [ ]:
# 训练 GANITE
print("开始训练 GANITE...")
print("="*60)

ganite_model = GANITE(input_dim=X.shape[1], hidden_dim=64)
history = train_ganite(
    ganite_model, X, T, Y,
    n_epochs=200,
    batch_size=128,
    lr_g=1e-3,
    lr_d=1e-3,
    alpha=2.0,
    verbose=True
)

print("="*60)
print("训练完成！")

---

## Part 4: 评估与可视化

In [ ]:
# 评估模型
def evaluate_ganite(model: GANITE, X: np.ndarray, true_tau: np.ndarray) -> Dict:
    """
    评估 GANITE 性能
    """
    model.eval()
    with torch.no_grad():
        X_tensor = torch.FloatTensor(X)
        ite_pred = model.predict_ite(X_tensor).numpy().squeeze()
    
    # PEHE
    pehe = np.sqrt(np.mean((true_tau - ite_pred) ** 2))
    
    # ATE 误差
    ate_true = np.mean(true_tau)
    ate_pred = np.mean(ite_pred)
    ate_error = np.abs(ate_true - ate_pred)
    
    # ITE 相关性
    ite_corr = np.corrcoef(true_tau, ite_pred)[0, 1]
    
    return {
        'pehe': pehe,
        'ate_true': ate_true,
        'ate_pred': ate_pred,
        'ate_error': ate_error,
        'ite_corr': ite_corr,
        'ite_pred': ite_pred
    }

# 评估
ganite_metrics = evaluate_ganite(ganite_model, X, true_tau)

print("GANITE 评估结果")
print("="*60)
print(f"PEHE: {ganite_metrics['pehe']:.4f}")
print(f"ATE 真实值: {ganite_metrics['ate_true']:.4f}")
print(f"ATE 预测值: {ganite_metrics['ate_pred']:.4f}")
print(f"ATE 误差: {ganite_metrics['ate_error']:.4f}")
print(f"ITE 相关性: {ganite_metrics['ite_corr']:.4f}")

In [ ]:
# 可视化
fig = make_subplots(rows=2, cols=2,
                    subplot_titles=('训练损失 - 反事实块', '训练损失 - ITE块',
                                   'ITE 预测 vs 真实', 'ITE 预测误差分布'))

# 反事实块损失
fig.add_trace(go.Scatter(y=history['cf_g_loss'], name='G_cf Loss', 
                         line=dict(color=COLORS['primary'])), row=1, col=1)
fig.add_trace(go.Scatter(y=history['cf_d_loss'], name='D_cf Loss',
                         line=dict(color=COLORS['danger'])), row=1, col=1)

# ITE 块损失
fig.add_trace(go.Scatter(y=history['ite_g_loss'], name='G_ite Loss',
                         line=dict(color=COLORS['success'])), row=1, col=2)
fig.add_trace(go.Scatter(y=history['ite_d_loss'], name='D_ite Loss',
                         line=dict(color=COLORS['warning'])), row=1, col=2)

# ITE 散点图
fig.add_trace(go.Scatter(x=true_tau, y=ganite_metrics['ite_pred'],
                         mode='markers', marker=dict(size=3, opacity=0.5, color=COLORS['info']),
                         name='预测'), row=2, col=1)
fig.add_trace(go.Scatter(x=[true_tau.min(), true_tau.max()],
                         y=[true_tau.min(), true_tau.max()],
                         mode='lines', line=dict(color='red', dash='dash'),
                         name='理想线'), row=2, col=1)

# 误差分布
ite_error = ganite_metrics['ite_pred'] - true_tau
fig.add_trace(go.Histogram(x=ite_error, nbinsx=50, name='误差分布',
                           marker_color=COLORS['primary']), row=2, col=2)

# 更新坐标轴标签
fig.update_xaxes(title_text="Epoch", row=1, col=1)
fig.update_xaxes(title_text="Epoch", row=1, col=2)
fig.update_xaxes(title_text="真实 ITE", row=2, col=1)
fig.update_xaxes(title_text="预测误差", row=2, col=2)

fig.update_yaxes(title_text="损失", row=1, col=1)
fig.update_yaxes(title_text="损失", row=1, col=2)
fig.update_yaxes(title_text="预测 ITE", row=2, col=1)
fig.update_yaxes(title_text="频数", row=2, col=2)

fig.update_layout(height=700, title_text='GANITE 训练与评估结果', template='plotly_white')
fig.show()

---

## Part 5: GANITE vs TARNet/DragonNet 对比

### 方法对比表

| 维度 | TARNet | DragonNet | GANITE |
|------|--------|-----------|--------|
| **核心思想** | 共享表示 + 双头 | 三头 + 倾向得分 | GAN 生成反事实 |
| **损失函数** | Factual Loss | Factual + PS + TR | 对抗 + 监督 |
| **反事实处理** | 预测 $\mu_0, \mu_1$ | 预测 $\mu_0, \mu_1$ | 生成分布 |
| **不确定性** | 无 | 有限 | 可通过多次采样 |
| **训练难度** | 简单 | 中等 | 复杂（GAN 不稳定）|
| **数据需求** | 中等 | 中等 | 较大 |
| **适用场景** | 通用 | 强混淆 | 复杂分布 |

### GANITE 的独特优势

1. **生成式建模**：不是简单预测均值，而是生成整个分布
2. **不确定性量化**：通过多次采样可以得到预测区间
3. **灵活性**：可以处理更复杂的结果分布

### GANITE 的局限

1. **训练不稳定**：GAN 的老问题
2. **模式崩溃**：可能生成重复的反事实
3. **计算复杂**：两个 GAN 块需要更多计算资源

---

## 思考题参考答案

### 1. 为什么 GANITE 用 GAN 而不是 VAE 来生成反事实？

**参考答案**:

GAN 和 VAE 各有优劣，GANITE 选择 GAN 的原因：

**GAN 的优势**:
1. **更锐利的生成**：GAN 生成的样本通常更清晰，不会像 VAE 那样模糊
2. **模式覆盖**：对于复杂的多峰分布，GAN 理论上能更好地生成各种模式
3. **无需显式密度**：不需要建模 p(Y_cf|X,T,Y)，只需判别真假

**VAE 的优势** (CEVAE用的就是VAE):
1. **训练稳定**：VAE 有明确的ELBO目标，训练更稳定
2. **不确定性量化自然**：通过后验分布自然获得不确定性
3. **无模式崩溃**：VAE不会有GAN的模式崩溃问题

**对比表**:

| 维度 | GAN (GANITE) | VAE (CEVAE) |
|------|-------------|-------------|
| 生成质量 | 更锐利 | 较模糊 |
| 训练稳定性 | 较差 | 较好 |
| 不确定性 | 需多次采样 | 自然获得 |
| 计算成本 | 高（两个网络对抗）| 中等 |
| 模式崩溃 | 可能发生 | 不会 |

**GANITE选择GAN的理由**:
- 反事实推断需要生成"看起来真实"的结果
- GAN的对抗训练恰好符合这个要求
- 对于医疗等高风险场景，生成质量比训练稳定性更重要

**实践建议**:
- 数据量大、计算资源充足 → GANITE (GAN)
- 训练稳定性重要、需要不确定性 → CEVAE (VAE)
- 两者可以结合：用VAE学隐变量，用GAN生成反事实

---

### 2. GANITE 的两阶段设计有什么好处？

**参考答案**:

两阶段设计是GANITE的核心创新，有以下好处：

**阶段分离的好处**:

1. **问题分解**:
   - Block 1: 专注于"填补缺失的反事实"
   - Block 2: 专注于"从完整数据估计ITE"
   - 每个阶段解决一个明确的子问题

2. **训练稳定性**:
   - 一次训练一个GAN比同时训练多个任务更稳定
   - Block 1先收敛，为Block 2提供稳定输入

3. **模块化设计**:
   - 可以单独改进每个模块
   - 可以用其他方法替换某个块（如用VAE替换Block 1）

**如果只用一个GAN块**:

```python
# 单阶段设计（不推荐）
class SingleStageGAN:
    # 直接从 X 生成 ITE
    ITE_hat = Generator(X, Z)
    
    # 判别器很难设计：
    # - 如何判断ITE是否"真实"？
    # - 缺少ground truth的监督信号
```

**问题**:
- 没有中间监督（反事实填补）
- 判别器难以设计（ITE本身不可观测）
- 训练更不稳定

**两阶段的协同**:

```
Block 1: 
输入 (X,T,Y) → 生成 Y_cf → 获得完整 (Y0, Y1)
         ↓
    有监督信号 (观测的Y)

Block 2:
输入 完整(Y0, Y1) → 估计 ITE = Y1 - Y0
         ↓
    有监督信号 (Block 1的输出)
```

**结论**: 两阶段设计是必要的，它将困难问题分解为两个可训练的子问题。

---

### 3. 如果真实的 ITE 分布是多峰的，GANITE 能捕捉到吗？

**参考答案**:

**理论上**：GANITE 能够捕捉多峰ITE分布

**原因**:
1. **GAN的生成能力**：GAN擅长生成复杂的多峰分布
2. **随机噪声Z**：Block 1中的Z让生成器能产生多样性
3. **对抗训练**：鼓励生成器覆盖真实分布的所有模式

**举例 - 多峰ITE场景**:

```python
# 想象一个药物治疗效果依赖基因型
# 基因型A: ITE ≈ +5 (有效)
# 基因型B: ITE ≈ 0  (无效)
# 基因型C: ITE ≈ -3 (有害)

true_ite_distribution = Mixture([
    Normal(+5, 1),  # 峰1
    Normal(0, 0.5), # 峰2
    Normal(-3, 1)   # 峰3
])
```

**GANITE vs TARNet**:

| 方法 | 多峰分布 | 原因 |
|------|---------|------|
| TARNet | ❌ 只能预测均值 | $\mu_0(x), \mu_1(x)$ 是确定性函数 |
| DragonNet | ❌ 只能预测均值 | 同TARNet |
| CEVAE | ✅ 可以 | VAE建模分布 |
| GANITE | ✅ 可以 | GAN生成分布 |

**但实践中的挑战**:

1. **模式崩溃**：GAN可能只学到一个峰
2. **需要足够数据**：每个峰都需要足够样本
3. **评估困难**：如何验证是否捕捉到所有峰？

**验证方法**:

```python
# 多次采样ITE
ite_samples = []
for _ in range(100):
    ite = model.predict_ite(x)
    ite_samples.append(ite)

# 检查分布
plt.hist(np.concatenate(ite_samples), bins=50)
# 应该看到多个峰
```

**结论**: GANITE理论上能捕捉多峰ITE，但实践中需要：
- 足够的数据
- 仔细调参避免模式崩溃
- 通过多次采样验证

---

### 4. GANITE 的判别器 D_cf 判断的是什么？

**参考答案**:

D_cf 的任务是：**判断给定的 (X, T, Y_factual, Y_cf) 组合是否"合理"**

**详细分析**:

输入：$(X, T, Y_{factual}, Y_{cf})$

D_cf需要回答：
- 如果 T=1：给定X和观测到的Y(1)，生成的Y(0)看起来像真的吗？
- 如果 T=0：给定X和观测到的Y(0)，生成的Y(1)看起来像真的吗？

**判别器学习的模式**:

1. **结果的合理范围**:
   - Y_cf 的值是否在合理区间内？
   - 例如：健康评分不能<0或>100

2. **X和Y的一致性**:
   - 给定患者特征X，Y_cf是否符合预期？
   - 例如：年轻患者的治疗效果通常更好

3. **Y(0)和Y(1)的关系**:
   - 两个潜在结果的差异是否合理？
   - 例如：治疗效应不太可能>1000%

**与生成器的博弈**:

```
Generator想: "我生成的Y_cf要让D_cf认为是真的"
       ↓
   生成更逼真的反事实

Discriminator想: "我要分辨哪些是生成的"
       ↓
   学习真实反事实的特征
```

**监督信号的关键作用**:

注意GANITE有监督损失：

$$L_G = L_{adv} + \alpha \cdot L_{supervised}$$

其中 $L_{supervised} = (Y - \hat{Y}_{factual})^2$

这确保：
- 生成器不会完全脱离现实
- 即使判别器失败，监督损失仍然约束生成器

**类比**:

想象D_cf是一个医生：
- 给他看一个患者X和治疗后结果Y(1)
- 告诉他："如果这个患者没治疗，会是什么结果？"
- 他根据经验判断生成器给出的Y(0)是否合理

**结论**: D_cf通过学习X→Y的映射模式，帮助G_cf生成更符合真实分布的反事实。

---

### 5. 在医疗场景中，GANITE 的不确定性量化有什么实际意义？

**参考答案**:

在高风险医疗决策中，不确定性量化至关重要！

**实际应用场景**:

#### 场景1: 个性化治疗决策

```python
# 对患者 A 预测ITE
ite_samples = []
for _ in range(100):
    y0, y1 = ganite.generate_counterfactual(x_patient_A)
    ite_samples.append(y1 - y0)

ite_mean = np.mean(ite_samples)
ite_std = np.std(ite_samples)
ite_95ci = np.percentile(ite_samples, [2.5, 97.5])

print(f"预期ITE: {ite_mean:.2f}")
print(f"不确定性: {ite_std:.2f}")
print(f"95%置信区间: [{ite_95ci[0]:.2f}, {ite_95ci[1]:.2f}]")
```

**决策规则**:

```python
if ite_mean > 0 and ite_95ci[0] > 0:
    decision = "强烈推荐治疗（高信心）"
elif ite_mean > 0 but ite_95ci[0] < 0:
    decision = "建议治疗，但需监测（中等信心）"
elif ite_mean > 0 but ite_std > threshold:
    decision = "效果不确定，建议更多检查"
else:
    decision = "不推荐治疗"
```

#### 场景2: 风险评估

```python
# 计算治疗有害的概率
p_harmful = (np.array(ite_samples) < 0).mean()

if p_harmful > 0.1:
    print(f"警告：有 {p_harmful:.1%} 的概率治疗有害")
    # 触发额外的安全检查
```

#### 场景3: 临床试验设计

使用不确定性指导样本选择：

```python
# 选择最不确定的患者进行RCT
uncertainty_scores = ite_stds
topk_uncertain = np.argsort(uncertainty_scores)[-100:]

print(f"建议对这{len(topk_uncertain)}名患者进行RCT")
# 这样可以最高效地减少不确定性
```

**为什么重要**:

1. **医患沟通**:
   - "这个治疗对您有70%的概率有效"比"应该有效"更诚实
   - 帮助患者理解风险

2. **法律责任**:
   - 医生需要informed consent
   - 不确定性量化提供了决策依据

3. **资源优化**:
   - 对高不确定性患者做更多检查
   - 对高信心患者快速决策

4. **持续学习**:
   - 高不确定性 = 模型不确定 = 需要更多数据
   - 指导数据收集策略

**GANITE vs 确定性模型**:

| 模型 | 输出 | 医疗决策 |
|------|------|---------|
| TARNet | ITE = 3.2 | "治疗效果是3.2" |
| GANITE | ITE ~ N(3.2, 1.5) | "治疗效果约3.2，但不确定性±1.5" |

**真实案例启发**:

- **IBM Watson for Oncology**: 曾因过度自信的推荐被质疑
- **如果有不确定性量化**: "推荐化疗（70%信心）vs 不推荐（30%信心）"更负责任

**最佳实践**:

```python
def make_treatment_decision(patient_data):
    # 1. 预测ITE分布
    ite_dist = ganite.predict_ite_distribution(patient_data, n_samples=1000)
    
    # 2. 计算决策指标
    expected_benefit = ite_dist.mean()
    uncertainty = ite_dist.std()
    p_benefit = (ite_dist > 0).mean()
    p_large_benefit = (ite_dist > 0.5).mean()
    
    # 3. 综合决策
    if p_large_benefit > 0.8:
        return "强烈推荐", "high confidence"
    elif p_benefit > 0.6 and uncertainty < 2.0:
        return "推荐", "medium confidence"
    elif uncertainty > 3.0:
        return "需要更多检查", "high uncertainty"
    else:
        return "不推荐", "medium confidence"
```

**结论**: 不确定性量化让AI辅助医疗决策更加负责任、透明和可信。

---

*这些答案展示了GANITE的深度和在实际应用中的价值！* 🎓

## 总结

| 概念 | 定义 | 重要性 |
|------|------|--------|
| **反事实生成** | 用 GAN 生成「从未发生」的结果 | GANITE 核心创新 |
| **两阶段设计** | 先填补反事实，再估计 ITE | 分离问题，降低难度 |
| **对抗训练** | G 和 D 的博弈 | 保证生成质量 |
| **监督损失** | 利用观测值约束生成 | 防止偏离真实 |

### 核心公式回顾

**反事实生成器**：
$$\tilde{Y}_{cf} = G_{cf}(X, T, Y, Z)$$

**ITE 生成器**（确定性）：
$$\hat{\tau}(X) = G_{ite}(X)$$

### 何时使用 GANITE？

- ✅ 需要不确定性量化
- ✅ 结果分布复杂（非高斯）
- ✅ 有足够的数据和计算资源
- ❌ 数据量小
- ❌ 需要快速迭代

---

## 延伸阅读

- **原论文**: Yoon, J., Jordon, J., & van der Schaar, M. (2018). GANITE: Estimation of Individualized Treatment Effects using Generative Adversarial Nets. ICLR 2018.
- **相关工作**: CEVAE, TEDVAE, CausalGAN

---

*恭喜你完成了 GANITE 的学习！* 🎉